In [2]:
%matplotlib inline 
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import _pickle
import seaborn as sns
import matplotlib.pyplot as plt
import folium
from folium.plugins import HeatMap

In [3]:
with open('pkls/K_2_weights.pkl','rb') as f:
    k_2 = _pickle.load(f)
with open('pkls/k_1.pkl', 'rb') as f:
    k_1 = _pickle.load(f)
with open('pkls/time_index.pkl', 'rb') as f:
    time_index = _pickle.load(f)
with open('pkls/mesh_index.pkl', 'rb') as f:
    mesh_index = _pickle.load(f)
with open('pkls/cat_index.pkl', 'rb') as f:
    cat_index = _pickle.load(f)
with open('pkls/tsdiv_list.pkl', 'rb') as f:
    tsdiv_list = _pickle.load(f)

In [4]:
print(k_2['U'].shape, k_2['V'].shape, k_2['W'].shape)

(2, 62) (2, 215) (2, 27)


In [5]:
# U:time, V:region, W:category
df_2U = pd.DataFrame(k_2['U'].transpose(), index=tsdiv_list, columns=['1', '2'])
df_2V = pd.DataFrame(k_2['V'].transpose(), index=mesh_index, columns=['1', '2'])
df_2W = pd.DataFrame(k_2['W'].transpose(), index=cat_index, columns=['1', '2'])

In [6]:
import re  # 正規表現によるパターンマッチ

def meshcode_to_latlng(code, loc = 'C'):

	if type(code) != int:  # 整数型に（文字列が渡された場合に対応）
		try:
			code = int(code)
		except:  # 形式チェック
			print("Error in  mesh_lib.meshocode_to_latlong()  Not a valid code handed.")
			raise

	code = str(code) # あらためて文字型に。

	code12 = ''
	code34 = ''
	code5  = ''
	code6  = ''
	code7  = ''
	code8  = ''

	loc = loc.upper()

	if re.match("^C|(NE)|(NW)|(SE)|(SW)$", loc) == None:
		raise Exception ("Invalid option to meshcode_to_latlong()")

	match_1_result = re.match("\d{4}", code)

	if re.match("\d{4}", code):  # 最初の４文字が数字

		code12 = code[0:2]
		code34 = code[2:4]
		lat_width  = 2.0 / 3.0  # grid cell の緯度方向の間隔
		long_width = 1.0        # grid cell の経度方向の間隔

	else:
		return (None)           # メッシュコードとして無効

	if re.match("\d{6}", code):   # 少なくとも最初の６文字は数字

		code5 = code[4:5]
		code6 = code[5:6]
		lat_width  /= 8.0;
		long_width /= 8.0;

	if re.match("\d{8}", code):  # 最初の８文字は数字

		code7 = code[6:7]
		code8 = code[7:8]
		lat_width  /= 10.0;
		long_width /= 10.0;

    # 以下、南西コーナーの座標を求める。

	lat  = float(code12) * 2 / 3          #  一次メッシュ
	long = float(code34) + 100

	if (code5 != '') & (code6 != ''):     # 二次メッシュ or 三次メッシュ
		lat  += (float(code5) * 2 / 3) / 8
		long += float(code6) / 8 

	if (code7 != '') & (code8 != ''):     # 三次メッシュ
		lat  += float(code7) * 2 / 3 / 8 / 10
		long += float(code8) / 8 / 10 

	# ここまでで南東端の緯度・経度が得られた

	# 中央の座標なら、一区画の幅（経度幅）・高さ（緯度幅）の半分を足す。
	if loc == 'C':
		lat  += lat_width  / 2 
		long += long_width / 2

	# 北端の座標なら、一区画の高さ（緯度幅）を足す
	if re.search('N', loc):
		lat += lat_width

	# 東端の座標なら、一区画の幅（経度幅）を足す
	if re.search('E', loc):
		long  += long_width

	return (lat, long)     # タプルを返す。

In [7]:
def add_mesh12_to_df(df):
    m12lat_list = []
    m12lng_list = []
    for index, row in df.iterrows():
        mesh3 = int(str(index)[:8])
        part_12 = int(str(index)[8:])
        m3lat, m3lng = meshcode_to_latlng(mesh3, 'SW')
        
        p_lat = 15 / 3600
        p_lng = 22.5 / 3600
        
        if part_12 == 1:
            m12lat = m3lat + p_lat
            m12lng = m3lng + p_lng
        elif part_12 == 2:
            m12lat = m3lat + p_lat
            m12lng = m3lng + 3 * p_lng
        elif part_12 == 3:
            m12lat = m3lat + 3 * p_lat
            m12lng = m3lng + p_lng
        elif part_12 == 4:
            m12lat = m3lat + 3 * p_lat
            m12lng = m3lng + 3 * p_lng
        
        m12lat_list.append(m12lat)
        m12lng_list.append(m12lng)
    
    df['lat'] = m12lat_list
    df['lng'] = m12lng_list
    return df

In [8]:
add_mesh12_to_df(df_2V)

,1,2,lat,lng
533903872,-0.305095,-0.886316,35.404167,139.48125
533903873,0.252710,-0.814711,35.412500,139.46875
533903874,0.728194,-0.843498,35.412500,139.48125
533903363,0.017442,-0.849436,35.370833,139.45625
533903364,-0.035079,-0.838285,35.370833,139.46875
533903361,-0.372437,-0.843060,35.362500,139.45625
533903881,-0.885186,-0.911954,35.404167,139.48125
533903371,0.049316,-0.891030,35.362500,139.46875
533903372,-0.758921,-0.795093,35.362500,139.48125
533903373,0.138795,-0.913533,35.370833,139.46875


In [9]:
# tf_zansa
def mk_heatmap(df, hm=None, max_zoom=16, radius=30, blur=15, min_opacity=None): # Make a heatmap
    fujisawa = [35.37,139.47]
    center = fujisawa
    hmap = folium.Map(location=center, zoom_start=12)
    marker_list = []
    for index, row in df.iterrows():
        if index == 533903871:
            print(row[hm])
        if hm == None:
#             marker_latlng = [df.at[df.index.values[i],'lat'], df.at[df.index.values[i],'lng']]
            marker_latlng = [row['lat'], row['lng']]
            input_list = marker_latlng
        else:
#             marker_latlngx = [df.at[df.index.values[i],'lat'], df.at[df.index.values[i],'lng'], df.loc[df.index.values[i], hm]]
            marker_latlngx = [row['lat'], row['lng'], row[hm] * 10]
            input_list = marker_latlngx
        marker_list.append(input_list)
    if hm == None:
        hmarker = HeatMap(marker_list)
    else:
        if min_opacity == None:
            hmarker = HeatMap(marker_list, max_zoom=max_zoom, radius=radius, blur=blur)
        else:    
            hmarker = HeatMap(marker_list, min_opacity=min_opacity, max_zoom=max_zoom, radius=radius, blur=blur)
    hmap.add_child(hmarker)
    return hmap

In [24]:
mk_heatmap(df_2V, '1', 16, 40, 20, 0.3)
# mk_heatmap(df_2V, '1', 15.6, 33, 16, 0.05)

0.012468742206692696


In [11]:
def output_bar2(K,L=15):
    # item_k graph
    list_k = np.array((k_item[K]))
    array_item = np.array(list_item)
    tmp = np.c_[array_item, list_k]
    df_k = pd.DataFrame(tmp, columns=['item','k'])
    df_k = df_k.sort_values(by='k', ascending=False)
    list_1to12 = np.array(np.arange(len(k_item.index)))
    df_1to12 = pd.DataFrame(list_1to12, index=df_k.index, columns=['x_label'])
    df_k = pd.concat([df_k, df_1to12], axis=1)
    x1 = list(range(k_item.index.shape[0]))
    y1 = np.array(df_k['k'].tolist())
#     y1 = np.sort(y1)[::-1]

    # mesh_k graph
    df_1 = np.array(matrix_mesh_k[K])
    df_2 = list(map(str, list(matrix_mesh_k.index)))
    df_latlng = pd.DataFrame(np.array([matrix_mesh_k['lat'],matrix_mesh_k['lng']]).T, columns=['lat', 'lng'])
    df_12 = pd.DataFrame(df_1, columns=['k'])
    df_22 = pd.DataFrame(df_2, columns=['code'])
    num_list = list(np.arange(len(df_12)))
    df_3 = pd.concat([df_22, df_12, df_latlng], axis=1)
    df_3 = df_3.sort_values(by='k', ascending=False)
    df_4 = pd.DataFrame(num_list, index=df_3.index, columns=['index'])
    df_4 = pd.concat([df_3,df_4], axis=1)
    y2 = list(df_4['k'])
#     y2 = np.sort(y2)[::-1]
    x_label = list(df_4.index)
    for i in range(len(y2)-L):
        y2 = np.delete(y2, len(y2)-1)
        x_label = np.delete(x_label, len(x_label)-1)
    x2 = list(range(len(y2)))    
    
    # plot
#     plt.rcParams['font.family'] = 'IPAPGothic' #全体のフォントを設定
#     plt.rcParams["figure.figsize"] = [20, 12] # グラフのサイズを指定
#     plt.rcParams['font.size'] = 20 #フォントサイズを設定 default : 12
#     plt.rcParams['xtick.labelsize'] = 30 # 横軸のフォントサイズ
#     plt.rcParams['ytick.labelsize'] = 30 # 縦軸のフォントサイズ
    
#     hm_tmp = sns.heatmap(a["Q"],xticklabels=list_item, yticklabels=['1', '2', '3', '4', '5', '6'], cmap='Greys')
#     hm_tmp.set_xticklabels(list_item, rotation=60)
    
    print('kind-total')
    kind_num_bar()
    
    print('kind-k')
    fig, ax = plt.subplots()
    plt.bar(x1, y1, tick_label=df_k.index)
    ax.set_xticklabels(df_k['item'], rotation=300)
    fname1 = 'r' + str(K) + '_kind2.eps'
    print(fname1)
    plt.savefig(fname1, bbox_inches="tight")
    plt.show()
    
    print('kind-k-dif')
    kind_dif_bar(matrix_kind_k_nor_dif, K, 0)
    
    print(dict_item)#dict(zip(list(np.arange(len(x1))),list_item)))
    
    print('area-k')
    plt.bar(x2, y2, tick_label=x_label)
    plt.show()
    
    # heatmap
#     mk_heatmap(df_sum_latlng, 'total', 17, 27, 20, 0.2)
    
    fujisawa = [35.37,139.47]
    center = [35.37,139.55]
    hmap = folium.Map(location=center, zoom_start=12)
    marker_list = []
    df = df_4
    for i in range(len(y2)):
#         if hm == None:
#             marker_latlng = [df.at[df.index.values[i],'lat'], df.at[df.index.values[i],'lng']]
#             input_list = marker_latlng
#         else:
        marker_latlngx = [df.at[df.index.values[i],'lat'], df.at[df.index.values[i],'lng'], df.loc[df.index.values[i], 'k']]
        input_list = marker_latlngx
        marker_list.append(input_list)
#     if hm == None:
#         hmarker = HeatMap(marker_list)
#     else:
#         hmarker = HeatMap(marker_list, max_zoom=13, radius=40, blur=18)
    hmarker = HeatMap(marker_list, max_zoom=13, radius=40, blur=18)
    hmap.add_child(hmarker)
    return hmap
    
    print(df_4.head(15))
    